In [ ]:
!pip install -q streamlit
!pip install supabase

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 842.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 2.2 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Hydration.csv to Hydration.csv


In [ ]:
%%writefile /content/app.py

import time
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from supabase import create_client
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

API_URL = 'API_KEY = 'TABLE_NAME = "Sigma GSR"
DATA_FILE = "Hydration.csv"
supabase = create_client(API_URL, API_KEY)

def load_training_data():
    df = pd.read_csv(DATA_FILE)
    return df

def train_model(df):
    le = LabelEncoder()
    df["Label_encoded"] = le.fit_transform(df["Hydration"])
    X = df[["MicroSiemens"]].values
    y = df["Label_encoded"].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, shuffle=True)
    model = RandomForestClassifier(n_estimators=10, max_depth=3, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return model, le, acc, cm

def fetch_gsr_data():
    response = supabase.table(TABLE_NAME).select("*").execute()
    data_list = response.data
    if not data_list:
        return pd.DataFrame(columns=["created_at", "GSR"])
    df_gsr = pd.DataFrame(data_list)
    df_gsr["created_at"] = pd.to_datetime(df_gsr["created_at"], errors='coerce')
    df_gsr["GSR"] = pd.to_numeric(df_gsr["GSR"], errors="coerce")
    df_gsr.sort_values(by="created_at", inplace=True)
    df_gsr.reset_index(drop=True, inplace=True)
    return df_gsr

def classify_latest_value(model, label_encoder):
    st.write("Waiting 5 seconds for latest GSR entry...")
    time.sleep(5)
    df_gsr_updated = fetch_gsr_data()
    if df_gsr_updated.empty:
        st.error("No GSR data found after waiting.")
        st.session_state["classification"] = None
        return
    latest_row = df_gsr_updated.iloc[-1]
    latest_gsr = latest_row["GSR"]
    latest_time = latest_row["created_at"]
    if latest_gsr < 7.12:
        pred_label = "Dehydrated"
    else:
        user_input = np.array([[latest_gsr]])
        pred_encoded = model.predict(user_input)[0]
        pred_label = label_encoder.inverse_transform([pred_encoded])[0]
    st.session_state["classification"] = {"time": latest_time, "gsr": latest_gsr, "label": pred_label}

def main():
    if "classification" not in st.session_state:
        st.session_state["classification"] = None
    page = st.sidebar.radio("Select Page", ["Machine Learning Dashboard", "Results"])
    df_train = load_training_data()
    model, label_encoder, acc, cm = train_model(df_train)
    if page == "Machine Learning Dashboard":
        st.title("Machine Learning Dashboard")
        st.subheader("Model Performance")
        st.write(f"Accuracy: {acc*100:.2f}%")
        st.write("Confusion Matrix:")
        st.write(cm)
        acc_percent = acc * 100.0
        acc_df = pd.DataFrame({"Metric": ["Accuracy"], "Value": [acc_percent]})
        fig_acc = px.bar(acc_df, x="Metric", y="Value", title="Random Forest Accuracy (%)", color="Metric", text="Value")
        fig_acc.update_traces(texttemplate='%{text:.2f}', textposition='outside')
        fig_acc.update_layout(yaxis_range=[0, 110])
        st.plotly_chart(fig_acc, use_container_width=True)
        cm_fig = px.imshow(cm, text_auto=True, color_continuous_scale='RdBu', title="Confusion Matrix")
        cm_fig.update_layout(width=500, height=400)
        st.plotly_chart(cm_fig, use_container_width=False)
        df_gsr = fetch_gsr_data()
        if df_gsr.empty:
            st.warning("No GSR data found in Supabase table.")
        else:
            st.subheader("Time vs. GSR (microSiemens)")
            fig_line = px.line(df_gsr, x="created_at", y="GSR", title="GSR Over Time", markers=True)
            st.plotly_chart(fig_line, use_container_width=True)
            st.subheader("Histogram: GSR Distribution")
            fig_hist = px.histogram(df_gsr, x="GSR", nbins=20, title="Histogram: GSR Distribution")
            st.plotly_chart(fig_hist, use_container_width=True)
        st.subheader("Classify the Latest GSR from Supabase")
        if st.button("Read & Classify from Supabase"):
            classify_latest_value(model, label_encoder)
            st.success("Classification complete! Go to 'Results' page.")
    else:
        st.title("Classification Results")
        classification = st.session_state["classification"]
        if classification is None:
            st.info("No classification performed yet. Go to 'Machine Learning Dashboard' to classify.")
        else:
            latest_time = classification["time"]
            latest_gsr = classification["gsr"]
            label = classification["label"]
            st.write(f"Latest Row: time={latest_time}, GSR={latest_gsr}")
            st.write(f"Classification: {label}")
            if label == "Dehydrated":
                st.error("You may be dehydrated – please drink water!")
                st.write("Dehydration can lead to:\n- Headaches, dizziness, and fatigue\n- Poor kidney function\n- Reduced concentration")
            else:
                st.success("You appear to be Hydrated!")
                st.write("Being well-hydrated can:\n- Improve brain function\n- Help your skin and digestion\n- Keep your energy levels up")
            st.write("Stay healthy and monitor your hydration regularly!")

if __name__ == "__main__":
    main()


Writing /content/app.py


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 3s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.148.22.22


In [ ]:
!npx localtunnel --port 8501

⠙your url is: https://dull-bags-rhyme.loca.lt
